In [93]:
import os
import copy
import time
import random
import numpy as np

import functools

from PIL import Image
from collections import namedtuple

import torch
import torch.nn as nn
import torch.optim as opt
import torch.nn.functional as F
import torch.utils.data as data

import torchvision
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from matplotlib import pyplot as plt

SEED = 241

In [27]:
def seed_everything(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)

  if torch.cuda.is_available(): 
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [28]:
from google.colab import drive
drive.mount('/content/drive')

if os.getcwd() != '/content/drive/My Drive':
  os.chdir(os.getcwd() + '/drive/MyDrive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
%%shell

DIR="PennFudanPed"
if [-d "$DIR"]; then
  echo "Installing in ${DIR}"
  unzip PennFudanPed.zip -d "${DIR}"
else
  echo "Create and installing in ${DIR}"
  mkdir "${DIR}"
  unzip PennFudanPed.zip -d "${DIR}"
fi

/bin/bash: line 2: [-d: command not found
Create and installing in PennFudanPed
mkdir: cannot create directory ‘PennFudanPed’: File exists
Archive:  PennFudanPed.zip
replace PennFudanPed/PennFudanPed/added-object-list.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace PennFudanPed/PennFudanPed/Annotation/FudanPed00001.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace PennFudanPed/PennFudanPed/Annotation/FudanPed00002.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace PennFudanPed/PennFudanPed/Annotation/FudanPed00003.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace PennFudanPed/PennFudanPed/PedMasks/FudanPed00001_mask.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace PennFudanPed/PennFudanPed/PedMasks/FudanPed00002_mask.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace PennFudanPed/PennFudanPed/PedMasks/FudanPed00003_mask.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace PennFudanPed/PennFudanPed/PNGImages/FudanPed00001.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
r

In [141]:
class PennFudanPedDataset(data.Dataset):

  def __init__(self, path):
    self.image_path = path + '/PNGImages/'
    self.mask_path = path + '/PedMasks/'
    self.images = os.listdir(os.getcwd() + self.image_path)
    self.masks = os.listdir(os.getcwd() + self.mask_path)

  def __getitem__(self, idx):
    image_path = os.getcwd() + self.image_path + self.images[idx]
    mask_path = os.getcwd() + self.mask_path + self.masks[idx]

    image = np.array(Image.open(image_path).convert('RGB'))
    mask = np.array(Image.open(mask_path))

    # instances are encoded as different colors
    obj_ids = np.unique(mask)
    # first id is the background, so remove it
    obj_ids = obj_ids[1:]

    # split the color-encoded mask into a set
    # of binary masks
    masks = mask == obj_ids[:, None, None]

    # get bounding box coordinates for each mask
    num_objs = len(obj_ids)
    boxes = []
    for i in range(num_objs):
        pos = np.where(masks[i])
        xmin = np.min(pos[1])
        xmax = np.max(pos[1])
        ymin = np.min(pos[0])
        ymax = np.max(pos[0])
        boxes.append([xmin, ymin, xmax, ymax])

    # convert everything into a torch.Tensor
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    # there is only one class
    labels = torch.ones((num_objs,), dtype=torch.int64)
    masks = torch.as_tensor(masks, dtype=torch.uint8)

    image_id = torch.tensor([idx])
    area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
    # suppose all instances are not crowd
    iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

    target = {}
    target["boxes"] = boxes
    target["labels"] = labels
    target["masks"] = masks
    target["image_id"] = image_id
    target["area"] = area
    target["iscrowd"] = iscrowd

    return image, target

  def __len__(self):
    return len(self.images)

In [142]:
batch_size = 1

dataset = PennFudanPedDataset('/PennFudanPed/PennFudanPed')
train_loader = data.DataLoader(dataset, batch_size=batch_size)
# dataset[0]

# From pretrained

In [143]:
# print(model)

In [144]:
predictor = models.detection.faster_rcnn.FastRCNNPredictor
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

num_classes = 2
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = predictor(in_features, num_classes)

In [145]:
# print(model)

# Modify backbone

In [146]:
backbone = models.mobilenet_v2(pretrained=True).features
backbone.out_channels = 1280
anchor_generator = models.detection.faster_rcnn.AnchorGenerator(sizes=(32, 64, 128, 256, 512), 
                                                                aspect_ratios=(0.5, 1.0, 2.0))

roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                                output_size=7,
                                                sampling_ratio=2)

model = models.detection.FasterRCNN(backbone, 
                                    num_classes=2,
                                    rpn_anchor_generator=anchor_generator, 
                                    box_roi_pool=roi_pooler)

# Get pretrained instance segmentation

In [147]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [160]:
model = get_instance_segmentation_model(num_classes=2)

In [161]:
device = torch.device('cpu')
optimizer = opt.Adam(model.parameters(), lr=1e-5)

model.train()

for batch in train_loader:
  optimizer.zero_grad()
  images, targets = batch
  images = list(image.permute(2, 0, 1).to(device) for image in images)
  targets = [{k: v[0].to(device) for k, v in targets.items()}]
  loss_dict = model(images, targets)
  losses = sum(loss for loss in loss_dict.values())
  losses.backward()
  optimizer.step()

torch.Size([2, 4])


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


torch.Size([1, 4])
torch.Size([1, 4])
